In [76]:
!pip install python-dotenv psycopg2

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 5.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp311-cp311-macosx_10_9_universal2.whl size=242138 sha256=68eb7c74d476fe1ef296d72f58b2804dc457a4b500bfa504cc8f3a72c02afce7
  Stored in directory: /Users/mr.chetsadakonkongphasuk/Library/Caches/pip/wheels/d9/83/60/e9660320860aef3c38a67dea6ff9538e4cad76502cb39ed280
Successfully built psycopg2

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [94]:
from supabase import create_client

url = "replace with your database URL."
key = "replace with your key."
supabase = create_client(url, key)

data = supabase.table("retail_store_sales").select("*").limit(12575).execute()
print(f"Total records retrieved: {len(data.data)}")

Total records retrieved: 1000


In [95]:
all_data = []
batch_size = 1000  # ดึงทีละ 1,000 records
offset = 0

while True:
    response = supabase.table("retail_store_sales").select("*").range(offset, offset + batch_size - 1).execute()

    if not response.data:  # ถ้าไม่มีข้อมูลเหลือแล้วให้หยุดดึง
        break

    all_data.extend(response.data)
    offset += batch_size  # ขยับไป batch ถัดไป

# แปลงเป็น DataFrame
df_supabase = pd.DataFrame(all_data)

In [100]:
df_supabase

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,Year_Week
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,1,2024-04-08
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,1,2023-07-17
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,0,2022-10-03
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,0,2022-05-02
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,0,2022-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...
12570,TXN_9347481,CUST_18,Patisserie,Item_23_PAT,38.0,4.0,152.0,Credit Card,In-store,2023-09-03,0,2023-08-28
12571,TXN_4009414,CUST_03,Beverages,Item_2_BEV,6.5,9.0,58.5,Cash,Online,2022-08-12,0,2022-08-08
12572,TXN_5306010,CUST_11,Butchers,Item_7_BUT,14.0,10.0,140.0,Cash,Online,2024-08-24,0,2024-08-19
12573,TXN_5167298,CUST_04,Furniture,Item_7_FUR,14.0,6.0,84.0,Cash,Online,2023-12-30,1,2023-12-25


In [101]:
import pandas as pd

def preprocess_sales_data_flexible(df, n_past_week=6, n_future_week=1, start_date=None, end_date=None):
    """
    Preprocesses the sales dataset to generate time-based features and future sales aggregation.

    Parameters:
    df (pd.DataFrame): The sales data
    n_past_week (int): Number of past weeks to use for features (default=6)
    n_future_week (int): Number of future weeks to predict aggregated sales (default=1)
    start_date (str): Optional start date for filtering data (format: 'YYYY-MM')
    end_date (str): Optional end date for filtering data (format: 'YYYY-MM')

    Returns:
    pd.DataFrame: Processed dataset with time-series features and future target sales
    """
    # Convert Transaction Date to datetime format
    df["Transaction Date"] = pd.to_datetime(df["Transaction Date"])

    # Extract Week (starting on Sunday) for aggregation
    df["Year_Week"] = df["Transaction Date"].dt.to_period("W-SUN").apply(lambda x: x.start_time.strftime('%Y-%m-%d'))

    # Convert Discount Applied to binary values (True=1, False=0)
    df["Discount Applied"] = df["Discount Applied"].fillna(False).astype(int)

    # Aggregate basic features
    grouped = df.groupby("Year_Week").agg(
        weekly_avg_repeat_purchases=("Customer ID", lambda x: x.duplicated().mean()),
        total_sales=("Total Spent", "sum"),
        unique_transactions=("Transaction ID", "nunique"),
        unique_customers=("Customer ID", "nunique"),
        discounted_transactions=("Discount Applied", "sum"),
    ).reset_index()

    # Aggregate category sales but shift to keep only t-1 to t-n values
    category_sales = df.groupby(["Year_Week", "Category"])["Total Spent"].sum().unstack(fill_value=0)
    # Shift to remove current week's data

    # Aggregate unique transactions per payment method
    payment_transactions = df.groupby(["Year_Week", "Payment Method"])["Transaction ID"].nunique().unstack(fill_value=0)

    # Aggregate unique transactions per location
    location_transactions = df.groupby(["Year_Week", "Location"])["Transaction ID"].nunique().unstack(fill_value=0)

    # Merge all aggregated data
    final_df = grouped.merge(category_sales, on="Year_Week", how="left") \
        .merge(payment_transactions, on="Year_Week", how="left") \
        .merge(location_transactions, on="Year_Week", how="left")

    # Fill NaNs resulting from merging
    # Allow NaN values for rows without data
    final_df.fillna(float('nan'), inplace=True)

    # Generate past weeks features (t-1 to t-N)
    feature_cols = [col for col in final_df.columns if col != "Year_Week"]
    past_week_cols = ["unique_transactions", "unique_customers", "discounted_transactions"] + feature_cols
    for col in feature_cols:
        for i in range(1, n_past_week + 1):
            final_df[f"{col}_t-{i}"] = final_df[col].shift(i)

    # Generate future target sales (T+1 to T+n_future_week)
    for i in range(1, n_future_week + 1):
        final_df[f"y_{i}"] = (
            final_df["total_sales"].shift(-i)
        )

    # Apply date filtering if specified
    if start_date:
        final_df = final_df[final_df["Year_Week"] >= start_date]
    if end_date:
        final_df = final_df[final_df["Year_Week"] <= end_date]

    return final_df


In [75]:
# data = pd.read_csv('data/retail_store_sales.csv')
# data

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
...,...,...,...,...,...,...,...,...,...,...,...
12570,TXN_9347481,CUST_18,Patisserie,Item_23_PAT,38.0,4.0,152.0,Credit Card,In-store,2023-09-03,NaN
12571,TXN_4009414,CUST_03,Beverages,Item_2_BEV,6.5,9.0,58.5,Cash,Online,2022-08-12,False
12572,TXN_5306010,CUST_11,Butchers,Item_7_BUT,14.0,10.0,140.0,Cash,Online,2024-08-24,NaN
12573,TXN_5167298,CUST_04,Furniture,Item_7_FUR,14.0,6.0,84.0,Cash,Online,2023-12-30,True


In [102]:
df_supabase["Discount Applied"] = df_supabase["Discount Applied"].replace({"True": True, "False": False}).fillna(False).astype(int)

In [105]:
data_prep = preprocess_sales_data_flexible(df_supabase,30,15)

/var/folders/mh/smhv27qx1n769y841db29nmr0000gn/T/ipykernel_16709/1926290959.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df[f"{col}_t-{i}"] = final_df[col].shift(i)
/var/folders/mh/smhv27qx1n769y841db29nmr0000gn/T/ipykernel_16709/1926290959.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df[f"{col}_t-{i}"] = final_df[col].shift(i)
/var/folders/mh/smhv27qx1n769y841db29nmr0000gn/T/ipykernel_16709/1926290959.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

In [112]:
print(list(data_prep.columns))

['Year_Week', 'weekly_avg_repeat_purchases', 'total_sales', 'unique_transactions', 'unique_customers', 'discounted_transactions', 'Beverages', 'Butchers', 'Computers and electric accessories', 'Electric household essentials', 'Food', 'Furniture', 'Milk Products', 'Patisserie', 'Cash', 'Credit Card', 'Digital Wallet', 'In-store', 'Online', 'weekly_avg_repeat_purchases_t-1', 'weekly_avg_repeat_purchases_t-2', 'weekly_avg_repeat_purchases_t-3', 'weekly_avg_repeat_purchases_t-4', 'weekly_avg_repeat_purchases_t-5', 'weekly_avg_repeat_purchases_t-6', 'weekly_avg_repeat_purchases_t-7', 'weekly_avg_repeat_purchases_t-8', 'weekly_avg_repeat_purchases_t-9', 'weekly_avg_repeat_purchases_t-10', 'weekly_avg_repeat_purchases_t-11', 'weekly_avg_repeat_purchases_t-12', 'weekly_avg_repeat_purchases_t-13', 'weekly_avg_repeat_purchases_t-14', 'weekly_avg_repeat_purchases_t-15', 'weekly_avg_repeat_purchases_t-16', 'weekly_avg_repeat_purchases_t-17', 'weekly_avg_repeat_purchases_t-18', 'weekly_avg_repeat_p

In [107]:
data_prep.to_csv("data/data_prep.csv")

In [124]:
from supabase import create_client

# ใส่ URL และ API Key ของ Supabase
url = "replace with your database URL."
key = "replace with your key."
supabase = create_client(url, key)

In [119]:
# สร้างตัวแปร SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS weekly_sales_data (
    Year_Week TEXT PRIMARY KEY,
    """ + ",\n    ".join([f'"{col}" FLOAT' for col in data_prep.columns if col != "Year_Week"]) + """
);
"""

# รันคำสั่ง SQL โดยใช้ execute_sql
# วิธีนี้ทำงานเฉพาะถ้า Supabase มีฟังก์ชัน execute_sql
response = supabase.rpc("execute_sql", {"query": create_table_query}).execute()

In [130]:
# ดูโครงสร้างคอลัมน์ของตาราง weekly_sales_data
get_columns_query = """
SELECT column_name 
FROM information_schema.columns 
WHERE table_schema = 'public' 
AND table_name = 'weekly_sales_data';
"""
response = supabase.rpc("execute_sql", {"query": get_columns_query}).execute()
columns = response.data
print(f"จำนวนคอลัมน์ทั้งหมด: {len(columns)}")
print("ตัวอย่างชื่อคอลัมน์ 10 แรก:", columns[:10])

# ดูชื่อคอลัมน์ใน DataFrame ของคุณ
print(f"คอลัมน์ใน DataFrame: {list(data_prep.columns)}")

จำนวนคอลัมน์ทั้งหมด: 0
ตัวอย่างชื่อคอลัมน์ 10 แรก: []
คอลัมน์ใน DataFrame: ['Year_Week', 'weekly_avg_repeat_purchases', 'total_sales', 'unique_transactions', 'unique_customers', 'discounted_transactions', 'Beverages', 'Butchers', 'Computers and electric accessories', 'Electric household essentials', 'Food', 'Furniture', 'Milk Products', 'Patisserie', 'Cash', 'Credit Card', 'Digital Wallet', 'In-store', 'Online', 'weekly_avg_repeat_purchases_t-1', 'weekly_avg_repeat_purchases_t-2', 'weekly_avg_repeat_purchases_t-3', 'weekly_avg_repeat_purchases_t-4', 'weekly_avg_repeat_purchases_t-5', 'weekly_avg_repeat_purchases_t-6', 'weekly_avg_repeat_purchases_t-7', 'weekly_avg_repeat_purchases_t-8', 'weekly_avg_repeat_purchases_t-9', 'weekly_avg_repeat_purchases_t-10', 'weekly_avg_repeat_purchases_t-11', 'weekly_avg_repeat_purchases_t-12', 'weekly_avg_repeat_purchases_t-13', 'weekly_avg_repeat_purchases_t-14', 'weekly_avg_repeat_purchases_t-15', 'weekly_avg_repeat_purchases_t-16', 'weekly_avg_repea

In [135]:
import numpy as np
import pandas as pd
import json

# 1. ลบตารางเดิมและสร้างใหม่
drop_table_query = "DROP TABLE IF EXISTS weekly_sales_data;"
supabase.rpc("execute_sql", {"query": drop_table_query}).execute()

# 2. กำหนดรายชื่อคอลัมน์ที่ถูกต้อง
columns = ['Year_Week', 'weekly_avg_repeat_purchases', 'total_sales', 'unique_transactions', 'unique_customers', 'discounted_transactions', 'Beverages', 'Butchers', 'Computers and electric accessories', 'Electric household essentials', 'Food', 'Furniture', 'Milk Products', 'Patisserie', 'Cash', 'Credit Card', 'Digital Wallet', 'In-store', 'Online', 'weekly_avg_repeat_purchases_t-1', 'weekly_avg_repeat_purchases_t-2', 'weekly_avg_repeat_purchases_t-3', 'weekly_avg_repeat_purchases_t-4', 'weekly_avg_repeat_purchases_t-5', 'weekly_avg_repeat_purchases_t-6', 'weekly_avg_repeat_purchases_t-7', 'weekly_avg_repeat_purchases_t-8', 'weekly_avg_repeat_purchases_t-9', 'weekly_avg_repeat_purchases_t-10', 'weekly_avg_repeat_purchases_t-11', 'weekly_avg_repeat_purchases_t-12', 'weekly_avg_repeat_purchases_t-13', 'weekly_avg_repeat_purchases_t-14', 'weekly_avg_repeat_purchases_t-15', 'weekly_avg_repeat_purchases_t-16', 'weekly_avg_repeat_purchases_t-17', 'weekly_avg_repeat_purchases_t-18', 'weekly_avg_repeat_purchases_t-19', 'weekly_avg_repeat_purchases_t-20', 'weekly_avg_repeat_purchases_t-21', 'weekly_avg_repeat_purchases_t-22', 'weekly_avg_repeat_purchases_t-23', 'weekly_avg_repeat_purchases_t-24', 'weekly_avg_repeat_purchases_t-25', 'weekly_avg_repeat_purchases_t-26', 'weekly_avg_repeat_purchases_t-27', 'weekly_avg_repeat_purchases_t-28', 'weekly_avg_repeat_purchases_t-29', 'weekly_avg_repeat_purchases_t-30', 'total_sales_t-1', 'total_sales_t-2', 'total_sales_t-3', 'total_sales_t-4', 'total_sales_t-5', 'total_sales_t-6', 'total_sales_t-7', 'total_sales_t-8', 'total_sales_t-9', 'total_sales_t-10', 'total_sales_t-11', 'total_sales_t-12', 'total_sales_t-13', 'total_sales_t-14', 'total_sales_t-15', 'total_sales_t-16', 'total_sales_t-17', 'total_sales_t-18', 'total_sales_t-19', 'total_sales_t-20', 'total_sales_t-21', 'total_sales_t-22', 'total_sales_t-23', 'total_sales_t-24', 'total_sales_t-25', 'total_sales_t-26', 'total_sales_t-27', 'total_sales_t-28', 'total_sales_t-29', 'total_sales_t-30', 'unique_transactions_t-1', 'unique_transactions_t-2']

# 3. กำหนดประเภทข้อมูลของแต่ละคอลัมน์
integer_columns = ["unique_transactions", "unique_customers", "discounted_transactions", "Cash", "Credit Card", "Digital Wallet", "In-store", "Online"]
text_columns = ["Year_Week"]

# 4. สร้างคำสั่ง SQL สำหรับสร้างตาราง
create_table_query = 'CREATE TABLE weekly_sales_data (\n'

# เพิ่มคอลัมน์แต่ละคอลัมน์ด้วยประเภทข้อมูลที่เหมาะสม
for col in columns:
    if col == "Year_Week":
        create_table_query += f'    "{col}" TEXT PRIMARY KEY,\n'
    elif col in integer_columns:
        create_table_query += f'    "{col}" INTEGER,\n'
    else:
        create_table_query += f'    "{col}" FLOAT,\n'

# ลบ comma ตัวสุดท้ายและปิด query
create_table_query = create_table_query.rstrip(',\n') + '\n);'

# 5. บันทึกคำสั่ง SQL ลงในไฟล์เพื่อใช้ในกรณีที่ต้องการรันผ่าน SQL Editor
with open("create_weekly_sales_table.sql", "w") as f:
    f.write(create_table_query)
print("บันทึกคำสั่ง SQL ลงในไฟล์ create_weekly_sales_table.sql แล้ว")

# 6. รันคำสั่งสร้างตาราง
try:
    response = supabase.rpc("execute_sql", {"query": create_table_query}).execute()
    print("สร้างตารางสำเร็จ")
except Exception as e:
    print(f"เกิดข้อผิดพลาดในการสร้างตาราง: {str(e)[:200]}...")
    print("โปรดใช้ SQL Editor ใน Supabase Dashboard เพื่อรันคำสั่ง SQL ที่ได้บันทึกไว้")

# 7. เพิ่มข้อมูลด้วยวิธีแปลง DataFrame เป็น JSON
try:
    # จัดการค่า float ที่ไม่ถูกต้อง
    data_clean = data_prep.copy()
    data_clean = data_clean.replace([np.inf, -np.inf], np.nan)

    # เลือกเฉพาะคอลัมน์ที่ต้องการ (กรณีที่ columns ไม่ครบตามที่มีใน data_prep)
    columns_to_use = [col for col in columns if col in data_clean.columns]
    data_filtered = data_clean[columns_to_use]

    # แปลง DataFrame เป็น JSON แล้วกลับมาเป็น Python native types
    data_json = data_filtered.to_json(orient='records')
    data_list = json.loads(data_json)

    # แทรกข้อมูลเป็นแบตช์
    batch_size = 5  # แบ่งเป็นชุดละ 5 แถว
    total_batches = (len(data_list) + batch_size - 1) // batch_size
    successful_rows = 0
    failed_rows = 0

    for batch_idx in range(total_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(data_list))
        batch_data = data_list[start_idx:end_idx]

        try:
            supabase.table("weekly_sales_data").insert(batch_data).execute()
            successful_rows += len(batch_data)
            print(f"เพิ่มข้อมูลแบตช์ {batch_idx + 1}/{total_batches} สำเร็จ ({successful_rows} แถว)")
        except Exception as e:
            print(f"เกิดข้อผิดพลาดในแบตช์ {batch_idx + 1}: {str(e)[:200]}...")
            # ถ้าแบตช์ล้มเหลว ลองแทรกทีละแถว
            for i, row_data in enumerate(batch_data):
                try:
                    supabase.table("weekly_sales_data").insert(row_data).execute()
                    successful_rows += 1
                    print(f"  - เพิ่มแถว {start_idx + i + 1} สำเร็จ")
                except Exception as row_error:
                    failed_rows += 1
                    if failed_rows <= 3:
                        print(f"  - เกิดข้อผิดพลาดที่แถว {start_idx + i + 1}: {str(row_error)[:100]}...")

    print(f"สรุป: เพิ่มข้อมูลสำเร็จ {successful_rows} แถว, ล้มเหลว {failed_rows} แถว")

    # 8. ตรวจสอบผลลัพธ์
    if successful_rows > 0:
        # ตรวจสอบจำนวนแถวที่เพิ่มเข้าไปในตาราง
        count_query = "SELECT COUNT(*) FROM weekly_sales_data;"
        response = supabase.rpc("execute_sql", {"query": count_query}).execute()
        print(f"จำนวนแถวทั้งหมดในตาราง: {response.data}")

        # ดึงตัวอย่าง 5 แถวแรก
        sample_query = "SELECT * FROM weekly_sales_data LIMIT 5;"
        response = supabase.rpc("execute_sql", {"query": sample_query}).execute()
        print("ตัวอย่าง 5 แถวแรก:")
        for row in response.data:
            print(row)

except Exception as e:
    print(f"เกิดข้อผิดพลาดในการเพิ่มข้อมูล: {str(e)[:200]}...")

บันทึกคำสั่ง SQL ลงในไฟล์ create_weekly_sales_table.sql แล้ว
สร้างตารางสำเร็จ
เกิดข้อผิดพลาดในแบตช์ 1: {}...
  - เกิดข้อผิดพลาดที่แถว 1: {}...
  - เกิดข้อผิดพลาดที่แถว 2: {}...
  - เพิ่มแถว 3 สำเร็จ
  - เพิ่มแถว 4 สำเร็จ
  - เพิ่มแถว 5 สำเร็จ
เพิ่มข้อมูลแบตช์ 2/32 สำเร็จ (8 แถว)
เพิ่มข้อมูลแบตช์ 3/32 สำเร็จ (13 แถว)
เพิ่มข้อมูลแบตช์ 4/32 สำเร็จ (18 แถว)
เพิ่มข้อมูลแบตช์ 5/32 สำเร็จ (23 แถว)
เพิ่มข้อมูลแบตช์ 6/32 สำเร็จ (28 แถว)
เพิ่มข้อมูลแบตช์ 7/32 สำเร็จ (33 แถว)
เพิ่มข้อมูลแบตช์ 8/32 สำเร็จ (38 แถว)
เพิ่มข้อมูลแบตช์ 9/32 สำเร็จ (43 แถว)
เพิ่มข้อมูลแบตช์ 10/32 สำเร็จ (48 แถว)
เพิ่มข้อมูลแบตช์ 11/32 สำเร็จ (53 แถว)
เพิ่มข้อมูลแบตช์ 12/32 สำเร็จ (58 แถว)
เพิ่มข้อมูลแบตช์ 13/32 สำเร็จ (63 แถว)
เพิ่มข้อมูลแบตช์ 14/32 สำเร็จ (68 แถว)
เพิ่มข้อมูลแบตช์ 15/32 สำเร็จ (73 แถว)
เพิ่มข้อมูลแบตช์ 16/32 สำเร็จ (78 แถว)
เพิ่มข้อมูลแบตช์ 17/32 สำเร็จ (83 แถว)
เพิ่มข้อมูลแบตช์ 18/32 สำเร็จ (88 แถว)
เพิ่มข้อมูลแบตช์ 19/32 สำเร็จ (93 แถว)
เพิ่มข้อมูลแบตช์ 20/32 สำเร็จ (98 แถว)
เพิ่มข้อมูลแบตช์ 21/32 สำ